In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import numpy as np
import seaborn as sns
import datetime
from sklearn import preprocessing
import sweetviz as sv
import os
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from matplotlib import pyplot
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from joblib import Parallel, delayed
import multiprocessing
from functools import partial
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
pd.set_option('display.max_columns', 70)
from sklearn.preprocessing import StandardScaler


# Loading Data:
3 different data is loaded in this analysis("ad06" , "defects" and "shipped+parcel"). 

In [ ]:
# reading ad06 parquet file
data_ad06 = pd.read_parquet('20211207_ad06_GATECH.parquet',engine='pyarrow')

In [ ]:
data_ad06.head()

Many columns are specified by steelcase to be droped. 

In [ ]:
# reading ad06 brincl cols to drop or keep file
xlsx = pd.ExcelFile("OneDrive_1_2-15-2022/ad06 brincl cols to drop or keep.xlsx")
drop_data = pd.read_excel(xlsx,"ad06 cols to drop or keep")

#reading defects data
defects = pd.read_excel("Defects.xlsx")
defects.head()

In [ ]:

#reading Shipped+parcel data
path = os.getcwd()
files = os.listdir(path+"/Pieces Shipped + Product Hierarchy 2-21-2022/2020")
y2020 = pd.DataFrame()
for f in files:
    data = pd.read_excel("Pieces Shipped + Product Hierarchy 2-21-2022/2020/"+f)
    y2020 = y2020.append(data)
    
    

files = os.listdir(path+"/Pieces Shipped + Product Hierarchy 2-21-2022/2021")
y2021 = pd.DataFrame()
for f in files:
    data = pd.read_excel("Pieces Shipped + Product Hierarchy 2-21-2022/2021/"+f)
    y2021 = y2021.append(data)
    

files = os.listdir(path+"/Pieces Shipped + Product Hierarchy 2-21-2022/2022")
y2022 = pd.DataFrame()
for f in files:
    data = pd.read_excel("Pieces Shipped + Product Hierarchy 2-21-2022/2022/"+f)
    y2022 = y2022.append(data)

data_connection = pd.concat([y2020,y2021,y2022])
data_connection.head()

# Data Prepration:

Dropping extra columns from ad06 data:

In [ ]:
#list of drop columns 
drop_data["Drop?"].fillna("NO",inplace=True)
drop_columns = drop_data[drop_data['Drop?'].str.contains("Yes")]["Field"]

#dropping columns from ad60
data_ad06_clean = data_ad06.drop(list(drop_columns),axis=1)

filtering defects data

In [ ]:
#filter value stream = "Components" in defects data
defects_filtered = defects[defects["Value Stream / Vendor"]=="Components"]
defects_filtered = defects_filtered.reset_index(drop = True)

In [ ]:
# defects_filtered['Cause Code Concat '].unique()
# cause_code = ['1250 - Missing','1150 - Wrong Style','1151 - Wrong Finish']
# defects_filtered = defects_filtered.loc[defects_filtered['Cause Code Concat '].isin(['1250 - Missing','1150 - Wrong Style','1151 - Wrong Finish'])]
# defects_filtered = defects_filtered.reset_index(drop = True)
# defects_filtered.shape

filtering shipped+parcel data

In [ ]:
#filter value stream = "Components" in shipped+parcel data
data_connection_filtered = data_connection[data_connection["Value Stream"]=="Components"]
data_connection_filtered = data_connection_filtered.reset_index(drop = True)

In ad06 data we need to change some columns format from str to int to be able to do merge with other data

In [ ]:
#convert some str column to int in ad06
data_ad06_clean_int = data_ad06_clean[~data_ad06_clean["4ZSD_AD06_VBELN"].str.contains('[A-Za-z]')]
data_ad06_clean_int['4ZSD_AD06_VBELN'] = data_ad06_clean_int['4ZSD_AD06_VBELN'].map(lambda x:int(x))
data_ad06_clean_int['4ZSD_AD06_POSNR'] = data_ad06_clean_int['4ZSD_AD06_POSNR'].map(lambda x:int(x))
data_ad06_clean_int['4ZSD_AD06_UEPOS'] = data_ad06_clean_int['4ZSD_AD06_UEPOS'].map(lambda x:int(x))
data_ad06_clean_int["concat"] = data_ad06_clean_int[['4ZSD_AD06_VBELN','4ZSD_AD06_POSNR']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

There are some duplicate in shipped+parcel data since some orders were shipped in more than one part, we need to aggrefate them:

In [ ]:
#removing date column from shipp data to group them based on other columns
connection_col = list(data_connection_filtered)
connection_col.remove("Act Ship Com Dt")
group_col = connection_col.copy()
group_col.remove("Pieces Shipped")
data_connection_grouped = data_connection_filtered[connection_col].groupby(group_col).sum().reset_index()
data_connection_grouped["concat"] = data_connection_grouped[["Sales Order","Sales Order item"]].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)


Merge "ad06" and "shipped+parcel" data

In [ ]:
ad06_connection = data_ad06_clean_int.merge(data_connection_grouped,on =["concat"] )
ad06_connection["style"] = ad06_connection[["4ZSD_AD06_POSNR","4ZSD_AD06_UEPOS"]].apply(lambda x: x["4ZSD_AD06_UEPOS"] if x["4ZSD_AD06_UEPOS"]!=0 else x["4ZSD_AD06_POSNR"],
                                    axis=1)


In [ ]:
merge_style = ad06_connection.copy()
merge_style["claim"] = "NO"
for row in defects_filtered.iterrows():
    merge_style.loc[(merge_style["4ZSD_AD06_VBELN"] == row[1]["Sales Order"]) & (merge_style["style"] == row[1]["Sales Order item"]),"claim"] = "Yes"


In [ ]:
print(merge_style["claim"].value_counts())
print(merge_style["claim"].value_counts()/len(merge_style))

In [ ]:
merge_drop_col = ['4ZSD_AD06_VBELN',
 '4ZSD_AD06_POSNR',
 '4ZSD_AD06_UEPOS',
 '0PLANT',
 '0PLANT___T',
 '4ZSD_AD06_MATNR',
 '4ZSD_AD06_KONDM', 
 '4ZSD_AD06_ZZTUSPCLIND',
 'GM Category',
 'concat',
 'Sales Order',
 'Sales Order item',    
 'Value Stream',
 'Reporting Plant',
 'Unnamed: 12',
 'style',
 'Material (Parcel)',
 '4ZSD_AD06_ERDAT',
 '4ZSD_AD06_ZZCONF_DELV',
 'Product Sub-Cat',  
 '4ZSD_AD06_ZZCONF_MAD',
 '4ZSD_AD06_ZZCUSTREQDATE']

merge_clean = merge_style.drop(list(merge_drop_col),axis=1)

merge_clean_head = merge_clean.head()

num_col = ['4ZSD_AD06_BRGEW',
 '4ZSD_AD06_KBMENG',
 '4ZSD_AD06_NTGEW',
 '4ZSD_AD06_VOLUM',
 '4ZSD_AD06_ZZLINEVOLUME',
 '4ZSD_AD06_ZZLINEWEIGHT',
 'Pieces Shipped']

cat_col = ['4ZSD_AD06_MATKL',
 '4ZSD_AD06_VSTEL',
 '4ZSD_AD06_ZZCARTIND',
 '4ZSD_AD06_ZZCONPATH',
 'PCC',
 'Product Line',
 'Op Prod Category',
 'PL Category',
 'Work Center']



for c in cat_col:
    print('\n------',c,'--------')
    print(merge_clean[c].value_counts())

In [ ]:
merge_clean["4ZSD_AD06_ZZCARTIND"].fillna("U",inplace=True)

# threshold = 3


# for feature in cat_col:
#     if len(merge_clean[feature].unique()) > 4:
#         print(feature)
#         index, counts = np.unique(merge_clean[feature].values,return_counts=True)
#         df_temp = pd.DataFrame({feature:index,"count":counts})
#         df_temp['percent'] = (df_temp['count'] / len(merge_clean)) * 100
#         keep_ = df_temp.loc[df_temp['percent'] >= threshold , feature]
#         merge_clean[feature] = merge_clean[feature].apply(lambda x:x if x in list(keep_) else "others")
 

In [ ]:
class data_preparation(object):
    def __init__(self,data):
        if len(data) == 0:
            return('Data Is Empty !!!')
        self.data = data
        
    def cat_aggregation(self, cols):
        def condense_category(col, min_freq=0.05, new_name='other'):
            series = pd.value_counts(col)
            mask = (series/series.sum()).lt(min_freq)
            return pd.Series(np.where(col.isin(series[mask].index), new_name, col))

        for c in cols:
            self.data[c] = condense_category(self.data[c], min_freq=0.05, new_name='other')
            print('-------','Aggregated',c,'--------')
            print(self.data[c].value_counts())
            
        return self.data   
    
              
    def scaler(self, num_cols):
        
        # pipeline for numerical variables
        # Filling missing values for numerical variable with the median of the data. Then, standardize the numerical variables.

        scaler = StandardScaler()
        num_var_fitted  = scaler.fit_transform(self.data[num_cols])
        num_var_fitted = pd.DataFrame(num_var_fitted, columns=num_cols)
        
        data = self.data.drop(num_cols,axis = 1)
        data = data.reset_index(drop = True)
        num_var_fitted = num_var_fitted.reset_index(drop = True)
        data = pd.concat([num_var_fitted, data], axis = 1)
        return data, scaler
    
   
    def OneHotEnc(self, cat_cols):
        # Column Transformer based on the defined pipeline for numerical data and filled categorical data which are 
        # ready to be inputed to one-hot-encoder function.
        for c in cat_cols:
            self.data[c] = self.data[c].astype(str)
        enc = OneHotEncoder()
        enc_fit = enc.fit_transform(self.data[cat_cols])
        col_names = enc.get_feature_names(cat_cols)

        enc_fit = enc_fit.toarray()
        enc_fit = pd.DataFrame(data = enc_fit, columns = col_names)
        
        data = self.data.drop(cat_cols,axis = 1)
        data = data.reset_index(drop = True)
        enc_fit = enc_fit.reset_index(drop = True)
        
        data = pd.concat([data,enc_fit], axis = 1)
        
        return data, col_names

                
        

In [ ]:
merge_clean = data_preparation(data = merge_clean).cat_aggregation(cat_col)

In [ ]:
merge_clean, cat_names = data_preparation(data = merge_clean).OneHotEnc(cat_col)
merge_clean.head()

In [ ]:

# down sampleing
claim = merge_clean[merge_clean["claim"] == "Yes"]
non_claim = merge_clean[merge_clean["claim"] == "NO"]
non_claim_downsample = resample(non_claim,
                         replace=True,
                         n_samples=len(claim),
                         random_state=42)
merge_clean_downsample = pd.concat([claim, non_claim_downsample]).reset_index(drop=True)

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(merge_clean_downsample, merge_clean_downsample["claim"]):
    ds_train_set = merge_clean_downsample.loc[train_index]
    ds_test_set = merge_clean_downsample.loc[test_index]


#check the ratio
print(ds_test_set["claim"].value_counts() / len(ds_test_set))
print(merge_clean_downsample["claim"].value_counts() / len(merge_clean_downsample))


ds_train_x = ds_train_set.loc[:, ds_train_set.columns != 'claim']
ds_train_y = ds_train_set["claim"]
ds_train_y[ds_train_y == "NO"] = 0
ds_train_y[ds_train_y == "Yes"] = 1
y_train = ds_train_y.astype('int')

ds_test_x = ds_test_set.loc[:, ds_test_set.columns != 'claim']
ds_test_y = ds_test_set["claim"]
ds_test_y[ds_test_y == "NO"] = 0
ds_test_y[ds_test_y == "Yes"] = 1
y_test = ds_test_y.astype('int')





In [ ]:
len(merge_clean_downsample)

In [ ]:
X_train, scaler = data_preparation(data = ds_train_x).scaler(num_col)
X_train.head()
# list(X_train)

In [ ]:
ds_test_x.head()

In [ ]:
X_test_scaled = scaler.transform(ds_test_x[num_col])
X_test_scaled = pd.DataFrame(X_test_scaled, columns=num_col)
X_test = ds_test_x.reset_index(drop = True)
X_test = pd.concat([X_test_scaled, X_test[cat_names]], axis = 1)
X_test.head()
X_test.shape

In [ ]:
X_train_feat_target = X_train.copy()
X_train_feat_target['claim'] = y_train
cor_matrix = X_train_feat_target.corr()
cor_matrix['claim'].sort_values(ascending = False)

# L1-based Linear SVM feature selection

In [ ]:
X = X_train
y = y_train

In [ ]:
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
df_prepared_reduced_l1_Based = model.transform(X)

In [ ]:
# shape of the reduced data
print('number of selected variables:', df_prepared_reduced_l1_Based.shape[1])
feature_idx = model.get_support()
feature_name = X.columns[feature_idx]
print('\nname of selected variables:', feature_name)
l1_based_features = list(feature_name)

# Lasso Based

In [ ]:
from sklearn import linear_model
lasso = linear_model.Lasso(alpha = 0.01).fit(X, y)
print('number of selected variables:', len(X.columns[lasso.coef_ > 0]))
print('\nname of selected variables:', X.columns[lasso.coef_ > 0])
lasso_based_features = list(X.columns[lasso.coef_ > 0])

# Tree-based feature selection

In [ ]:
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X, y)

In [ ]:
model = SelectFromModel(clf, prefit=True)
df_prepared_reduced_Tree_Based = model.transform(X)
df_prepared_reduced_Tree_Based.shape

In [ ]:
importances = clf.feature_importances_ #array with importances of each feature
features_to_keep = X.columns[importances > np.mean(importances)]
tree_based_features = list(features_to_keep)
print(features_to_keep)

In [ ]:
selected_features_dict = {'l1':l1_based_features,
                          'lasso':lasso_based_features,
                          'tree':tree_based_features
                         }

# Model Selection/Setup AND Hyperparameter Tuning

In [ ]:
from itertools import product
import json
def expand_grid(dictionary):
    return pd.DataFrame([row for row in product(*dictionary.values())], 
                       columns=dictionary.keys())

def create_model_library():
    # Heavy version
#     hyper_dict = {'KNN':{'n_neighbors':[3,5,7,9]},
#                'SVM':{'C':[0.5, 1, 5, 10],
#                  'gamma':[0.001,0.01,0.1,1,10]
#                 },
#                'RF':{'n_estimators':[20,40,60,100,200,300,400]},
#                'XGboost':{'n_estimators':[40,80,150,200,300],
#                      'max_depth':[3,4,5,6,8,10]
#                     },
#                'DL':{'batch_size':[8,16,32,64,128],
#                 'hidden_layer_sizes':[1,2,3,4]
#                },
#                'Adaboost':{'n_estimators':[10,40,80,150,200,300]},
#                'NaiveBayes':{'alpha':[0.001,0.01,0.1,1]},
#                'logisticReg':{'C':[0.001,0.01,0.1,0.25]}}
    
#     # light version
#     hyper_dict = {'KNN':{'n_neighbors':[3,5]},
#                'SVM':{'C':[0.5, 1],
#                  'gamma':[0.001,0.01,10]
#                 },
#                'RF':{'n_estimators':[20,40,60,100,200,300,400]},
#                'XGboost':{'n_estimators':[40,80,150],
#                      'max_depth':[3,4,5]
#                     },
#                'DL':{'batch_size':[32],
#                 'hidden_layer_sizes':[1,2,3,4]
#                },
#                'Adaboost':{'n_estimators':[10,40,80]},
#                'NaiveBayes':{'alpha':[0.001,1]},
#                'logisticReg':{'C':[0.1,0.25]}}
    
    # test version
    """
                   'SVM':{'C':[0.5, 1],
                 'gamma':[0.01,10]
                },
    """
    # first step is defing a dictionary of the selected models with their hyper parameters. 
    hyper_dict = {'KNN':{'n_neighbors':[3,5]},
               'RF':{'n_estimators':[20,100,50,200]},
               'XGboost':{'n_estimators':[40,150,300],
                     'max_depth':[3,4]
                    },
                'DL':{'batch_size':[32,64],
                 'hidden_layer_sizes':[1,2,3,4]
                },
               'Adaboost':{'n_estimators':[10,80,50,100]},
               'NaiveBayes':{'alpha':[0.001,1]},
               'logisticReg':{'C':[0.25,0.5]}}
    
    # Three varible selection method have been applied to select the feutures with important prediction power.
    variable_selection = ['l1', 'lasso', 'tree']
    
    # Creat the output of this function, which is a Pandas DataFrame.
    model_library = pd.DataFrame()
    
    
    for model_name in hyper_dict.keys(): # for each of the classification models
        temp = pd.DataFrame() # create a DataFrame which later will be concatinated to the model_library
        params = hyper_dict[model_name] # find the defined hyperparameter for that specific model
        
        # 'expand_grid' expands each of the hyperparameters.
        #(If 3 values are defined for a hyperparameter, three rows will be added to the model_library)
        temp['parameters'] = expand_grid(params).to_dict(orient='records') 

        temp['method_id'] = model_name
        model_wise_library = pd.DataFrame()
        for var_sel in variable_selection: # For each of the variable selection approaches, define a unique scenario. 
            temp['variable_selection_method'] = var_sel
            model_wise_library = pd.concat([model_wise_library, temp]) # Each of the scenarios will be concatinated.
            
        # For each classification model, obtained scenarios will be concatinated to model_library.
        model_library = pd.concat([model_wise_library, model_library]) 
    model_library['method_param_id'] = list(range(1, model_library.shape[0]+1)) # A unique ID for each scenario. 
    model_library = model_library.reset_index(drop=True)
    return model_library

In [ ]:
model_library = create_model_library()
model_library

In [ ]:
def create_model_obj(model_lib, n_features):
    # based on the 'method_id', creates the method_name
    method_name = model_lib['method_id'].values[0]
    
    # define the hyperparameters for that specific scenario, which is inputed to this fucntion. It stores like a dictionary.
    params = model_lib.parameters.tolist()[0]
    params = json.loads(str(params).replace("'", '"'))


    if method_name=='KNN':

        model = KNeighborsClassifier(n_neighbors = params['n_neighbors'])

    elif method_name=='SVM':

        model = SVC(C = params['C'], gamma = params['gamma'], probability = True)

    elif method_name == 'RF':

        model = RandomForestClassifier(n_estimators = params['n_estimators'])

    elif method_name == "XGboost":

        model = GradientBoostingClassifier(learning_rate = 0.01,
                                                n_estimators = params['n_estimators'],
                                                max_depth = params['max_depth'])

    elif method_name == "DL":

        number_features = n_features

        hidden_layer_sizes = params['hidden_layer_sizes']

        if(hidden_layer_sizes == 1):

            hidden_layer = (max(int(number_features/2), 1), max(int(number_features/4),1))

        elif(hidden_layer_sizes == 2):

            hidden_layer = (max(int(number_features),1), max(int(number_features/2),1))

        elif(hidden_layer_sizes == 3):

            hidden_layer = (max(int(number_features/2),1), max(int(number_features/3),1), max(int(number_features/6),1))

        elif(hidden_layer_sizes == 4):

            hidden_layer = (max(int(number_features),1), max(int(number_features/2),1), max(int(number_features/4),1))

        model = MLPClassifier(hidden_layer_sizes = hidden_layer,
                                   batch_size = params['batch_size'], 
                                  max_iter = 400)

    elif method_name == "Adaboost":

        model = AdaBoostClassifier(n_estimators = params['n_estimators'], learning_rate = 0.01)

    elif method_name == "NaiveBayes":

        model = BernoulliNB(alpha = params['alpha'])

    elif method_name == "logisticReg":

        model =  LogisticRegression(C = params['C'])

    else:

        model = None 

    return model


In [ ]:
def run_parallel_computation(method_param_id,
                             model_library, 
                             selected_features_dict, 
                             X_train, 
                             Y_train,
                             X_test,
                             Y_test
                            ):
    # select a specific scenario defined as a classification model with unique hyperparameters and selected features. 
    method_wise_lib = model_library.loc[model_library.method_param_id == method_param_id, ]
    # retrieve the future selection method, among available three approaches.
    var_sel_method = method_wise_lib.variable_selection_method.tolist()[0]
    # select the feature selection approach based on the designed dictionary of the three available approaches.
    imp_vars = selected_features_dict[var_sel_method]
    # creat the actual model based on the classification model, hyperparameter value, feature selection approach (stored in method_wise_lib)
    # and number of features.
    model = create_model_obj(method_wise_lib, len(imp_vars))
    # store the data for selected features
    X_train_with_selected_vars = X_train[imp_vars]
    # fit the model
    model.fit(X_train_with_selected_vars, Y_train)
    # since AUC is the evaluation criteria, probability will be the output of the model.
    prob = model.predict_proba(X_test[imp_vars])
    score = roc_auc_score(Y_test, prob[:, 1])
    temp_output = {'method_param_id':[method_param_id],
                   'roc_auc_score':[score]
                  }
    temp_output = pd.DataFrame(temp_output)
    return temp_output

In [ ]:
#parallel computation libraries
from joblib import Parallel, delayed
import multiprocessing
from functools import partial


num_cores = multiprocessing.cpu_count()

# keep in mind that 'method_param_id' is not set here since the paralleization is based on 'method_param_id'
partial_parallel = partial(run_parallel_computation,
                           model_library = model_library, 
                           selected_features_dict=selected_features_dict, 
                           X_train=X_train, 
                           Y_train=y_train,
                           X_test=X_test,
                           Y_test=y_test)

output = Parallel(n_jobs=num_cores)(delayed(partial_parallel)(i) for i in model_library.method_param_id.tolist())
results = pd.concat(output)

In [ ]:
sorted_result = results.sort_values(by = "roc_auc_score",ascending = False)
final_result = sorted_result.merge(model_library, on = "method_param_id",how = "inner")
final_result.head()



In [ ]:
best_model = GradientBoostingClassifier(learning_rate = 0.01,
                                         n_estimators = 300,
                                         max_depth = 4)
imp_vars = selected_features_dict["l1"]                                               
X_train_with_selected_vars = X_train[imp_vars]
best_model.fit(X_train_with_selected_vars, y_train)

In [ ]:
from matplotlib import pyplot
print(best_model.feature_importances_)
print(imp_vars)
pyplot.bar(imp_vars, best_model.feature_importances_)
pyplot.xticks(rotation=90)

In [ ]:
from matplotlib import pyplot
print(best_model.feature_importances_)
print(imp_vars)
pyplot.bar(imp_vars, best_model.feature_importances_)
pyplot.xticks(rotation=90)

In [ ]:
def plot_box( cols, col_x = 'claim'):
    for col in cols:
        sns.set_style("whitegrid")
        sns.boxplot(x = col_x, y = float(col), data = merge_clean_downsample)
        plt.xlabel(col_x) # Set label for the x axis
        plt.ylabel(col) # Set label for y axis
        plt.show()
plot_box(cols = ["4ZSD_AD06_ZZLINEVOLUME"])

In [ ]:
sns.set_style("whitegrid")
tmp_df = merge_clean_downsample[["4ZSD_AD06_ZZLINEVOLUME","claim"]]
tmp_df['4ZSD_AD06_ZZLINEVOLUME'] = pd.to_numeric(tmp_df['4ZSD_AD06_ZZLINEVOLUME'])
#tmp_df = tmp_df.loc[tmp_df['4ZSD_AD06_ZZLINEVOLUME']<200, ]
sns.boxplot(x = 'claim', y = '4ZSD_AD06_ZZLINEVOLUME', data = tmp_df)
plt.xlabel('claim') # Set label for the x axis
plt.ylabel('4ZSD_AD06_ZZLINEVOLUME') # Set label for y axis
plt.show()

In [ ]:
tmp_df.plot.scatter(x = 'claim', y = '4ZSD_AD06_ZZLINEVOLUME')

In [ ]:
work_center_data = merge_style[merge_style["claim"]=="Yes"]
work_center = (work_center_data["Work Center"].value_counts()/merge_style["Work Center"].value_counts())*100

In [ ]:
import seaborn as sns
sns.set_theme(style="darkgrid")
ax = sns.countplot(x="Work Center", data=work_center_data)
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

In [ ]:
sns.set_theme(style="darkgrid")
ax = sns.countplot(x="Work Center", data=merge_style)
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

In [ ]:
work_center.plot.bar()
